<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Table-of-Contents" data-toc-modified-id="Table-of-Contents-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Table of Contents</a></span></li></ul></div>

In [1]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder.
    appName("test").
    enableHiveSupport().
    config("spark.sql.warehouse.dir","/user/hive/warehouse").
    getOrCreate()
)
spark.sparkContext.getConf().getAll()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


[('spark.app.name', 'test'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.host', '192.168.199.77'),
 ('spark.driver.port', '10881'),
 ('spark.hadoop.fs.default.name', 'hdfs://localhost:9000'),
 ('spark.sql.warehouse.dir', '/user/hive/warehouse'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.startTime', '1638813659254'),
 ('spark.hadoop.fs.defaultFS', 'hdfs://localhost:9000'),
 ('spark.app.id', 'local-1638813659923'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [3]:
path = "file:///home/sergey/Py_SparkDataFrame_edx_CS105_CS110_CS120/data/201408_station_data.csv"
df = spark.read.csv(path, header=True, inferSchema = True)
df.show(5)

+----------+--------------------+---------+-----------+---------+--------+------------+
|station_id|                name|      lat|       long|dockcount|landmark|installation|
+----------+--------------------+---------+-----------+---------+--------+------------+
|         2|San Jose Diridon ...|37.329732|-121.901782|       27|San Jose|    8/6/2013|
|         3|San Jose Civic Ce...|37.330698|-121.888979|       15|San Jose|    8/5/2013|
|         4|Santa Clara at Al...|37.333988|-121.894902|       11|San Jose|    8/6/2013|
|         5|    Adobe on Almaden|37.331415|  -121.8932|       19|San Jose|    8/5/2013|
|         6|    San Pedro Square|37.336721|-121.894074|       15|San Jose|    8/7/2013|
+----------+--------------------+---------+-----------+---------+--------+------------+
only showing top 5 rows



In [5]:
df.printSchema()

root
 |-- station_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dockcount: integer (nullable = true)
 |-- landmark: string (nullable = true)
 |-- installation: string (nullable = true)



In [6]:
df.count()

70

In [7]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, DoubleType, DateType
from datetime import datetime as dt

In [9]:
to_int = F.udf(lambda x: int(x), IntegerType())
to_float = F.udf(lambda x: float(x), DoubleType())
to_date = F.udf(lambda x: dt.strptime(x, '%m/%d/%Y'), DateType())

In [10]:
data = (df.select(to_int(df['station_id']).alias('station_id')
              ,df['name']
              ,to_float(df['lat']).alias('lat')
              ,to_float(df['long']).alias('long')
              ,to_int(df['dockcount']).alias('dockcount')
              ,df['landmark']
              ,to_date(df['installation']).alias('installation')
              )
     )

In [11]:
data.count()

70

In [12]:
data.show(5)

+----------+--------------------+---------+-----------+---------+--------+------------+
|station_id|                name|      lat|       long|dockcount|landmark|installation|
+----------+--------------------+---------+-----------+---------+--------+------------+
|         2|San Jose Diridon ...|37.329732|-121.901782|       27|San Jose|  2013-08-06|
|         3|San Jose Civic Ce...|37.330698|-121.888979|       15|San Jose|  2013-08-05|
|         4|Santa Clara at Al...|37.333988|-121.894902|       11|San Jose|  2013-08-06|
|         5|    Adobe on Almaden|37.331415|  -121.8932|       19|San Jose|  2013-08-05|
|         6|    San Pedro Square|37.336721|-121.894074|       15|San Jose|  2013-08-07|
+----------+--------------------+---------+-----------+---------+--------+------------+
only showing top 5 rows



In [13]:
data.printSchema()

root
 |-- station_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dockcount: integer (nullable = true)
 |-- landmark: string (nullable = true)
 |-- installation: date (nullable = true)



In [15]:
data.createOrReplaceTempView('station')

In [17]:
spark.sql("select * from station where dockcount == 27").show()

+----------+--------------------+---------+-----------+---------+-------------+------------+
|station_id|                name|      lat|       long|dockcount|     landmark|installation|
+----------+--------------------+---------+-----------+---------+-------------+------------+
|         2|San Jose Diridon ...|37.329732|-121.901782|       27|     San Jose|  2013-08-06|
|        61|     2nd at Townsend|37.780526|-122.390288|       27|San Francisco|  2013-08-22|
|        67|      Market at 10th|37.776619|-122.417385|       27|San Francisco|  2013-08-23|
|        77|   Market at Sansome|37.789625|-122.400811|       27|San Francisco|  2013-08-25|
+----------+--------------------+---------+-----------+---------+-------------+------------+

